In [13]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.pyplot
import numpy as np
import cv2
import glob
import time
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [15]:
# Define a function to compute binned color features
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel()
    # Return the feature vector
    return features



In [16]:
# Define a function to compute color histogram features
def color_hist(img, nbins=32):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features   

In [17]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs (features and visulization) if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise, get features only
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [31]:

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()

def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                     hist_bins=32, orient=9,
                     pix_per_cell=8, cell_per_block=2, hog_channel=0,
                     spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else:
            feature_image = np.copy(image)

        if (spatial_feat == True):
            # Apply bin_spatial() to get spatial color features
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if (hist_feat == True):
            # Apply color_hist() also with a color space option now
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if (hog_feat == True):
            if (hog_channel == "All"):
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(
                        get_hog_features(feature_image[:, :, channel], orient, pix_per_cell, cell_per_block, vis=False,
                                         feature_vec=True))
                    hog_features = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(feature_image[:, :, hog_channel], orient,
                                                pix_per_cell, cell_per_block, vis=False, feature_vec=True)

            # Append the new feature vector to the features list
            file_features.append(hog_features)

        features.append(np.concatenate(file_features))
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features)))
    # Return list of feature vectors
    return features



In [32]:

# window size (x and y are dimension)
# and overlap fraction (for both x and y)

def slide_window(img,x_start,stop=[None,None],y_start_stop=[None,None], xy_window=(64,64), xy_overlap=(0.5,0.5)):
    # If x and/or y start/stop positions not defined, set to image size 
    if(x_start_stop[0] == None):
        x_start_stop[0] = 0 
    if(x_start_stop[1] == None):
        x_start_stop[1] = img.shape[1]
    if(y_start_stop[0] == None):
        y_start_stop[0] = 0 
    if(y_start_stop[1] == None):
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1-xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1-xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_windows = np.int(xspan/nx_pix_per_step) - 1
    ny_windows = np.int(yspan/ny_pix_per_step) - 1
    # Initialize a list to append window position to 
    
    window_list = [] 
    # Loop through finding x and y window positions 
    # Note: you could vectorize this step, but in practice 
    # you will be considering windows one by one with your classifier , so looping makes sense 
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs * nx_pix_per_step + x_start_stop[0]
            endex = startx + xy_window[0]
            starty = xs * ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
        
        # Append window position to list 
        window_list.append(((startx,starty),(endx,endy)))
        
    # Return the list of windows     
    return window_list


In [33]:

# Define a function to draw bounding boxes

def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the images
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        cv2.rectange(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with the boxes drawn
    return imcopy

   

In [34]:
# This function is very similar to extract_features()
# Just for a single image rather than the list of images.

def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True, vis=False):

    # Define an empty list to receive features
    img_features = []
    # Apply color conversion if other than RGB
    # apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else:
        feature_image = np.copy(img)

    if (spatial_feat == True):
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        img_features.append(spatial_features)
    if (hist_feat == True):
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins)
        img_features.append(hist_features)
    if hog_feat == True:
        if (hog_channel == "All"):
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(
                    get_hog_features(feature_image[:, :, channel], orient, pix_per_cell, cell_per_block,
                                     vis=False,feature_vec=True))
                hog_features = np.concatenate(hog_features)
        else:
            if (vis == True):
                hog_features, hog_image = get_hog_features(feature_image[:, :, hog_channel], orient,
                                                       pix_per_cell, cell_per_block, vis=True, feature_vec=True)            
            else:
                hog_features = get_hog_features(feature_image[:, :, hog_channel], orient,
                                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append features to list 
        img_features.append(hog_features)            

    if vis == True:
        return np.concatenate(img_features), hog_image
    else : 
        return np.concatenate(img_features)

        


In [35]:
  #   Define a function you will pass an image
    #   and the list of windows to be searched (output of slide_windows())

def search_windows(img,windows,clf,scaler,color_space='RGB',
                   spatial_size=(32,32),hist_bins=32,hist_range=(0,256),orient=9,
                   pix_per_cell=8,cell_per_block=2,hog_channel=0,spatial_feat=True,
                   hist_feat=True,hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_image = cv2.resize(img[window[0][1]:window[1][1],window[0][0]:window[1][0]],(64,64))
        #4) Extract features for that window using single img_feature()

        features = single_img_features(test_img,color_space=color_space,spatial_size=spatial_size,hist_bins=hist_bins,orient=orient,
                                       pix_per_cell=pix_per_cell,cell_per_block=cell_per_block,hog_channel=hog_channel,
                                       spatial_feat=spatial_feat,hist_feat=hist_feat,hog_feat=hog_feat)

        #5) Scale extracted to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1,-1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) if positive (prediction == 1) then save the window
        if(prediction ==1 ):
            on_windows.append(window)

    return on_windows


    

In [36]:
def visualize(fig,rows,cols,imgs,titles):
    for i, img in enumerate(imgs):
        plt.subplot(rows,cols,i+1)
        plt.title(i+1)
        img_dims = len(img.shape)        
        plt.imshow(img)                
        plt.title(titles[i])
    plt.show()

In [37]:
%matplotlib inline

cars = glob.glob('./vehicles/**/*.png')
notcars = glob.glob('./non-vehicles/**/*.png')

#Choose random car / not-car indices 
car_ind = np.random.randint(0,len(cars))
notcar_ind = np.random.randint(0,len(notcars))

# Read in car / not-car images
car_image = mpimg.imread(cars[car_ind])
notcar_image = mpimg.imread(notcars[notcar_ind])



# Define feature parametes 
color_space = 'RGB'# can be RGB, HSV, LUV, HLS, YUV , YCr
orient = 6
pix_per_cell = 8
cell_per_block = 2
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
spatial_size = (16,16) # Spatial binning dimensions 
hist_bins = 16
spatial_feat = True
hist_feat = True
hog_feat = True 
vis = True



In [38]:
car_features, car_hog_image = single_img_features(car_image, color_space, spatial_size,
                            hist_bins, orient,pix_per_cell, cell_per_block, hog_channel,
                            spatial_feat, 
                            hist_feat, 
                            hog_feat, 
                            vis)

notcar_features, notcar_hog_image = single_img_features(notcar_image, color_space, spatial_size,
                            hist_bins, orient, pix_per_cell, cell_per_block, hog_channel,
                            spatial_feat, hist_feat, hog_feat, vis)

images = [car_image,car_hog_image,notcar_image,notcar_hog_image]
titles = ['car image', 'car HOG image', 'notcar_image', 'notcar HOG image']
fig = plt.figure(figsize=(12,3))
visualize(fig, 1, 4, images, titles)

C:\ProgramData\Miniconda3\envs\carnd-term1\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [40]:
#Define feature parameters

color_space = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 6
pix_per_cell = 8
cell_per_block = 2
hog_channel = 0 # 'ALL'
spatial_size = (16,16)
hist_bins = 16
spatial_feat = True 
hist_feat = True 
hog_feat = True 

t = time.time()
n_samples = 1000
random_idxs = np.random.randint(0,len(cars),n_samples)
test_cars = cars
car_features = extract_features(test_cars,color_space,spatial_size,hist_bins,orient,pix_per_cell,cell_per_block,hog_channel,spatial_feat,hist_feat,hog_feat)

C:\ProgramData\Miniconda3\envs\carnd-term1\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
